In [ ]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
# from Imageio import Image
import tensorflow as tf
import imageio

%matplotlib inline

class CONFIG:
    ORIGIN_PATH = 'F:/houseworkData/neuralStyleTransfer/'
    IMAGE_WIDTH = 400
    IMAGE_HEIGHT = 300
    COLOR_CHANNELS = 3
    NOISE_RATIO = 0.6
    MEANS = np.array([123.68,116.779,103.939]).reshape((1,1,1,3))
    VGG_IMAGE = ORIGIN_PATH + 'imagenet-vgg-verydeep-19.mat'
    STYLE_IMAGE = ORIGIN_PATH + 'stone_style.jpg'
    CONTENT_IMAGE = ORIGIN_PATH + 'content300.jpg'
    OUTPUT_DIR = ORIGIN_PATH
    
STYLE_LAYERS = [
        ('conv1_1',0.2),
        ('conv2_1',0.2),
        ('conv3_1',0.2),
        ('conv4_1',0.2),
        ('conv5_1',0.2)
    ]
    
def load_vgg_model(path):
    vgg = scipy.io.loadmat(path)
    vgg_layers = vgg['layers']
    
    def _weights(layer, expected_layer_name):
        wb = vgg_layers[0][layer][0][0][2]
        W = wb[0][0]
        b = wb[0][1]
        layer_name = vgg_layers[0][layer][0][0][0][0]
        assert layer_name == expected_layer_name
        return W, b
    
    def _relu(conv2d_layer):
        return tf.nn.relu(conv2d_layer)

    def _conv2d(prev_layer, layer, layer_name):
        W, b = _weights(layer, layer_name)
        W = tf.constant(W)
        b = tf.constant(np.reshape(b,(b.size)))
        return tf.nn.conv2d(prev_layer, filter=W, strides=[1,1,1,1],padding='SAME') + b
    
    def _conv2d_relu(prev_layer, layer, layer_name):
        return _relu(_conv2d(prev_layer,layer,layer_name))
    
    def _avgpool(prev_layer):
        return tf.nn.avg_pool(prev_layer, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    graph = {}
    graph['input'] = tf.Variable(np.zeros((1,CONFIG.IMAGE_HEIGHT,CONFIG.IMAGE_WIDTH,CONFIG.COLOR_CHANNELS)), dtype='float32')
    graph['conv1_1'] = _conv2d_relu(graph['input'],0,'conv1_1')
    graph['conv1_2']=_conv2d_relu(graph['conv1_1'],2,'conv1_2')
    graph['avgpool1']=_avgpool(graph['conv1_2'])
    graph['conv2_1']=_conv2d_relu(graph['avgpool1'],5,'conv2_1')
    graph['conv2_2']=_conv2d_relu(graph['conv2_1'],7,'conv2_2')
    graph['avgpool2']=_avgpool(graph['conv2_2'])
    graph['conv3_1']=_conv2d_relu(graph['avgpool2'],10,'conv3_1')
    graph['conv3_2']=_conv2d_relu(graph['conv3_1'],12,'conv3_2')
    graph['conv3_3']=_conv2d_relu(graph['conv3_2'],14,'conv3_3')
    graph['conv3_4']=_conv2d_relu(graph['conv3_3'],16,'conv3_4')
    graph['avgpool3'] = _avgpool(graph['conv3_4'])
    graph['conv4_1']=_conv2d_relu(graph['avgpool3'],19,'conv4_1')
    graph['conv4_2']=_conv2d_relu(graph['conv4_1'],21,'conv4_2')
    graph['conv4_3']=_conv2d_relu(graph['conv4_2'],23,'conv4_3')
    graph['conv4_4']=_conv2d_relu(graph['conv4_3'],25,'conv4_4')
    graph['avgpool4'] = _avgpool(graph['conv4_4'])
    graph['conv5_1']=_conv2d_relu(graph['avgpool4'],28,'conv5_1')
    graph['conv5_2']=_conv2d_relu(graph['conv5_1'],30,'conv5_2')
    graph['conv5_3']=_conv2d_relu(graph['conv5_2'],32,'conv5_3')
    graph['conv5_4']=_conv2d_relu(graph['conv5_3'],34,'conv5_4')
    graph['avgpool5'] = _avgpool(graph['conv5_4'])
    return graph

def generate_noise_image(content_image,noise_ratio=CONFIG.NOISE_RATIO):
    noise_image = np.random.uniform(-20,20,(1,CONFIG.IMAGE_HEIGHT,CONFIG.IMAGE_WIDTH,CONFIG.COLOR_CHANNELS)).astype('float32')
    input_image = noise_image*noise_ratio+content_image*(1-noise_ratio)
    return input_image

def reshape_and_normalize_image(image):
    image = np.reshape(image,((1,)+image.shape))
    image = image - CONFIG.MEANS
    return image

def save_image(path, image):
    image = image + CONFIG.MEANS
    image = np.clip(image[0],0,255).astype('uint8')
    imageio.imsave(path, image)
    
# content cost
def compute_content_cost(a_C, a_G):
    m,n_H,n_W,n_C = a_G.get_shape().as_list()
    a_C_unrolled = tf.reshape(a_C,[n_H*n_W,n_C])
    a_G_unrolled = tf.reshape(a_G,[n_H*n_W,n_C])
#     print(m,n_H,n_W,n_C)
#     print(a_G.eval().shape)
#     print(a_G_unrolled.eval().shape)
    J_content = 1./(4*n_H*n_W*n_C)*tf.reduce_sum(tf.square(tf.subtract(a_C_unrolled, a_G_unrolled)))
    return J_content

# style matrix
def gram_matrix(A):
    GA = tf.matmul(A, tf.transpose(A))
    return GA

# style cost
def compute_layer_style_cost(a_S, a_G):
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    a_S = tf.transpose(tf.reshape(a_S, [n_H*n_W,n_C]))
    a_G = tf.transpose(tf.reshape(a_G, [n_H*n_W,n_C]))
    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)
    J_style_layer = 1./(4*n_C*n_C*n_H*n_W*n_H*n_W)*tf.reduce_sum(tf.square(tf.subtract(GS,GG)))
    return J_style_layer
    
def compute_style_cost(model, STYLE_LAYERS):
    J_style = 0
    for layer_name, coeff in STYLE_LAYERS:
        out = model[layer_name]
        a_S = sess.run(out)
        a_G = out
        J_style_layer = compute_layer_style_cost(a_S, a_G)
        J_style += coeff * J_style_layer
    return J_style

def total_cost(J_content, J_style, alpha=10, beta=40):
    J = alpha*J_content + beta*J_style
    return J
    
def model_nn(sess, input_image, num_iterations=200):
    sess.run(tf.global_variables_initializer())
    sess.run(model['input'].assign(input_image))
    
    for i in range(num_iterations):
        sess.run(train_step)
        generated_image = sess.run(model['input'])
        
        if i%20 == 0:
            Jt, Jc, Js = sess.run([J, J_content, J_style])
            print('iteration ', i, ':')
            print('total cost=', Jt)
            print('content cost=', Jc)
            print('style cost=', Js)
            save_image(CONFIG.ORIGIN_PATH+str(i)+'.png', generated_image)
    save_image(CONFIG.ORIGIN_PATH+'last.jpg', generated_image)
    

model = load_vgg_model(CONFIG.VGG_IMAGE)
#     print(model)

# tf.reset_default_graph()
sess = tf.InteractiveSession()

content_image = imageio.imread(CONFIG.ORIGIN_PATH+'louvre_small.jpg')
content_image = reshape_and_normalize_image(content_image)

style_image = imageio.imread(CONFIG.ORIGIN_PATH+'monet.jpg')
style_image = reshape_and_normalize_image(style_image)

generated_image = generate_noise_image(content_image)
#     imshow(generated_image[0])

sess.run(model['input'].assign(content_image))
out = model['conv4_2']
a_C = sess.run(out)
a_G = out
J_content = compute_content_cost(a_C, a_G)

sess.run(model['input'].assign(style_image))
J_style = compute_style_cost(model, STYLE_LAYERS)

J = total_cost(J_content, J_style, 10, 40)

optimizer = tf.train.AdamOptimizer(2.0)
train_step = optimizer.minimize(J)

model_nn(sess, generated_image)
sess.close()

d:\python35\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


iteration  0 :
total cost= 4946347500.0
content cost= 7864.7256
style cost= 123656720.0
